In [111]:
import pandas as pd

file_path = 'halflabelleddataset.csv'
df = pd.read_csv(file_path)

df['Sr. No'] = df['Sr. No'].fillna(method='ffill')
#df['Questions'] = df['Questions'].fillna('')
grouped_df = df.groupby('Sr. No')


required_columns = ['Data'] + [f'Unnamed: {i}' for i in range(3, 10)]
# Dictionary to store the processed DataFrames, keyed by 'Sr. No'
processed_groups = {}

for name, group in grouped_df:
    missing_columns = [col for col in required_columns if col not in group.columns]
    if missing_columns:
        print(f"Missing columns in group {name}: {missing_columns}")
        continue
    
    filtered_group = group[required_columns]
    filtered_group.fillna('', inplace=True)
    processed_groups[name] = filtered_group

df.drop(columns=required_columns, inplace=True)
new_df = df.groupby('Sr. No').first()

for i in range(1, len(processed_groups) + 1):
    d = processed_groups[i]
    new_columns = d.iloc[0]
    d.columns = new_columns
    d = d.drop(d.index[0])
    d.reset_index(drop=True, inplace=True)
    processed_groups[i] = d
    
new_df = new_df.assign(Data=processed_groups.values())
df = new_df

C:\Users\bavis\AppData\Local\Temp\ipykernel_3292\150138474.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\bavis\AppData\Local\Temp\ipykernel_3292\150138474.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('', inplace=True)
C:\Users\bavis\AppData\Local\Temp\ipykernel_3292\150138474.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_group.fillna('

In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

# Assuming df is your dataframe with columns Sr. No, Questions, Data, Alpha, Type
# Perform any necessary data preprocessing

df = df[df['Alpha'] != 'Insufficient Data']
# Split data into features and target variables
X_text = df['Questions']
X_numeric = df['Data']
y_alpha = df['Alpha']
y_type = df['Type']

# Vectorize text features
vectorizer = TfidfVectorizer()
X_text_vectorized = vectorizer.fit_transform(X_text)

# Combine text and numeric features
#X_combined = pd.concat([pd.DataFrame(X_text_vectorized.toarray()), X_numeric], axis=1)
#X_combined.drop(X_combined.tail(1).index,inplace=True) 
X_combined = X_text_vectorized

# Split data into training and testing sets
X_train, X_test, y_alpha_train, y_alpha_test, y_type_train, y_type_test = train_test_split(X_combined, y_alpha, y_type, test_size=0.2, random_state=42)

# Train separate models for Alpha and Type predictions
alpha_model = RandomForestClassifier()
alpha_model.fit(X_train, y_alpha_train)

type_model = RandomForestClassifier()
type_model.fit(X_train, y_type_train)

# Evaluate models
alpha_score = alpha_model.score(X_test, y_alpha_test)
type_score = type_model.score(X_test, y_type_test)

print(f"Alpha Model Score: {alpha_score}")
print(f"Type Model Score: {type_score}")


TypeError: train_test_split() got an unexpected keyword argument 'epochs'

In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

# Assuming df is your dataframe with columns Sr. No, Questions, Data, Alpha, Type
# Perform any necessary data preprocessing

df = df[df['Alpha'] != 'Insufficient Data']
# Split data into features and target variables
X_text = df['Questions']
X_numeric = df['Data']
y_alpha = pd.get_dummies(df['Alpha'])  # One-hot encoding for classification
y_type = pd.get_dummies(df['Type'])  # One-hot encoding for classification

# Vectorize text features
vectorizer = TfidfVectorizer()
X_text_vectorized = vectorizer.fit_transform(X_text)

# Combine text and numeric features
# For simplicity, we're only using vectorized text features in this example
X_combined = X_text_vectorized

# Split data into training and testing sets
X_train, X_test, y_alpha_train, y_alpha_test, y_type_train, y_type_test = train_test_split(X_combined, y_alpha, y_type, test_size=0.2, random_state=42)

# Neural network for Alpha
alpha_model = Sequential()
# One-hot encode the target variable
y_alpha_train_encoded = to_categorical(y_alpha_train)

# Fit the model with the one-hot encoded target variable
alpha_model.fit(X_train.toarray(), y_alpha_train_encoded, epochs=10, batch_size=32, validation_split=0.2)
type_model.add(Dense(y_type_train.shape[1], activation='softmax'))  # Output layer

type_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
type_model.fit(X_train.toarray(), y_type_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate models
alpha_loss, alpha_accuracy = alpha_model.evaluate(X_test.toarray(), y_alpha_test)
type_loss, type_accuracy = type_model.evaluate(X_test.toarray(), y_type_test)

print(f"Alpha Model Accuracy: {alpha_accuracy}")
print(f"Type Model Accuracy: {type_accuracy}")

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [113]:
X_text_sample = ["The retention scores (out of 10) of students using different study aids across various subjects Conduct a randomized block ANOVA to see if there are significant differences in retention scores at α = 0.05."]
X_text_vectorized_sample = vectorizer.transform(X_text_sample)

# Predict 'Alpha' values for the sample input
alpha_predictions_sample = alpha_model.predict(X_text_vectorized_sample)
print("Predicted Alpha:")
print(alpha_predictions_sample)

# Predict 'Type' values for the sample input
type_predictions_sample = type_model.predict(X_text_vectorized_sample)
print("Predicted Type:")
print(type_predictions_sample)

Predicted Alpha:
['0.05']
Predicted Type:
['One Way RBD']
